### Weather Training Data

In [45]:
import pandas as pd
from src.ForecastClient import ForecastClient

df = pd.read_csv('data/DRO_2025-01-01_2026-02-09.csv')

# basic data info
print("Data Size:  ", df.shape, "\n\n")
print("——— Headers ———")
print(*df.columns, "\n\n", sep=", ")

# load in sample forecast data
df_forecast = pd.read_json('data/forecast_example.json')

# print data to see what we may need to adjust
print("24 hours of each data point:")
for col in df.columns[2:]:
    if col not in df_forecast.columns:
        print(f"\n ***** Column '{col}' not found in forecast data. *****\n")
        continue
    print(pd.concat([df[col].head(24), df_forecast[col].head(24)], axis=1))

Data Size:   (9720, 24) 


——— Headers ———
name, datetime, temp, feelslike, dew, humidity, precip, precipprob, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, sunazimuth, sunelevation, stations, source, 


24 hours of each data point:
    temp  temp
0   18.1  23.6
1   18.1  25.4
2   16.8  26.8
3   12.4  25.2
4   12.4  20.9
5   12.8  24.7
6   13.4  23.3
7   17.8  21.5
8   20.0  25.6
9   21.6  35.5
10  26.4  43.1
11  30.2  47.0
12  34.7  51.0
13  38.0  53.0
14  41.0  54.9
15  40.6  56.0
16  40.6  54.9
17  36.4  54.1
18  28.4  47.9
19  25.3  45.0
20  23.3  42.0
21  20.4  40.9
22  18.9  38.9
23  21.7  38.0
    feelslike  feelslike
0         9.5       17.6
1         8.1       18.6
2        11.9       21.2
3        12.4       25.2
4        12.4       12.1
5         3.2       20.2
6        13.4       18.8
7        10.4       21.5
8        12.9       20.9
9        21.6       35.5
10       26.4       39.5

### ForcastClient Demonstration

In [2]:
forcast_data: list[dict] | None = None

try:
    fc = ForecastClient()
    forcast_data = fc.get_forecast(1)
except Exception as e:
    print(e)

if forcast_data:
    for fd in forcast_data:
        print(fd)

Getting forecast for 2026-02-11 days from 2026-02-10...
{'date': '2026-02-10', 'time': '00:00:00', 'datetime': '00:00:00', 'temp': 23.6, 'feelslike': 17.6, 'humidity': 56.37, 'dew': 10.4, 'precip': 0.0, 'precipprob': 0.0, 'snow': 0.0, 'snowdepth': 0.0, 'preciptype': None, 'windgust': 8.1, 'windspeed': 4.7, 'winddir': 32.0, 'visibility': 9.9, 'cloudcover': 0.0, 'solarradiation': 0.0, 'solarenergy': 0.0, 'uvindex': 0.0, 'sunazimuth': 344.3, 'sunelevation': -66.3, 'stations': ['TS761', 'KDRO', 'KCEZ', 'KFMN'], 'source': 'obs'}
{'date': '2026-02-10', 'time': '01:00:00', 'datetime': '01:00:00', 'temp': 25.4, 'feelslike': 18.6, 'humidity': 52.54, 'dew': 10.4, 'precip': 0.0, 'precipprob': 0.0, 'snow': 0.0, 'snowdepth': 0.0, 'preciptype': None, 'windgust': 8.1, 'windspeed': 5.7, 'winddir': 19.0, 'visibility': 9.9, 'cloudcover': 0.0, 'solarradiation': 0.0, 'solarenergy': 0.0, 'uvindex': 0.0, 'sunazimuth': 20.6, 'sunelevation': -65.8, 'stations': ['TS761', 'KDRO', 'KCEZ', 'KFMN'], 'source': 'obs

### Tyler's thoughts:

**Features I think will be most useful:**
- temp:  (hot == bad, so this is a good feature)
- precip engineered with preciptype (or just keep precip as is)
- snowdepth:  (increases ground albedo which increases bifacial production)
- windspeed:  (maybe — lowers cell temperature slightly, so more effective in summer)
- cloudcover:  (few clouds can actually increase irradiance, but a lot of clouds obviously blocks the sun, could be a fussy data point)
- solarradiation:  (this is kind of cheating cuz this is eastimated irradiance)
- solarenergy:  (related to solarradiation)
- sunazimuth:  (very important)
- sunelevation:  (also very important)